In [1]:
import numpy as np
from ngsolve import *
from netgen.occ import *
from netgen.geom2d import SplineGeometry
from ngsolve.internal import *
from xfem import *
import numpy as np
import ipywidgets
from ngsolve.solvers import *
from ngsolve.webgui import Draw

importing ngsxfem-2.1.2405


In [2]:

from ngsolve import *
import numpy as np

# ---------------- 0) Mesh & FE spaces ----------------
mesh  = Mesh("mesh.vol")

L     = H1(mesh, order=1)                         # phase φ space
Vh    = VectorH1(mesh, order=2)                   # velocity u space
Qh    = H1(mesh, order=1, dirichlet="Liq_Outlet_a")  # pressure
C     = H1(mesh, order=1, dirichlet="Gas_Inlet")  # CO2 space

phi   = GridFunction(L,  name="phi")
u     = GridFunction(Vh, name="u")
p     = GridFunction(Qh, name="p")
c     = GridFunction(C,  name="c")                # unknown CO2
c_old = GridFunction(C,  name="c_old")

# ---------------- 1) Load snapshots ----------------
U_data   = np.load("U_snap_freq60.5Hz.npy.npz")['arr_0']
Phi_data = np.load("Phi_snap_freq60.5Hz.npy.npz")['arr_0']


n_dof_p = p.vec.size
n_dof_u = u.vec.size
print("Snapshots:", U_data.shape[0], "  per-snapshot DOFs:", U_data.shape[1])
print("Vel DOFs:", n_dof_u, "  Press DOFs:", n_dof_p)

snapshot_idx = 120
phi.vec.FV().NumPy()[:] = Phi_data[snapshot_idx, :]
u.vec.FV().NumPy()[:]   = U_data[snapshot_idx, :n_dof_u]
p.vec.FV().NumPy()[:]   = U_data[snapshot_idx, n_dof_u:n_dof_u+n_dof_p]

# ---------------- 2) One-fluid ingredients ----------------
# Interface thickness ~ 2–3 element sizes
h_avg  = float(Integrate(specialcf.mesh_size, mesh)) / float(Integrate(1, mesh))
eps_if = Parameter(2.5*h_avg)

# Smooth Heaviside via logistic; then PROJECT to H1 so grad() is available
alpha_expr = 1.0 / (1.0 + exp(-phi/eps_if))
alphaL = GridFunction(L, name="alphaL")
alphaL.Set(alpha_expr)                         # projection/interpolation

tiny   = 1e-12
aL     = IfPos(alphaL - tiny, alphaL, tiny)
one_m  = IfPos(1-aL - tiny, 1-aL, tiny)
grad_aL = grad(alphaL)                         # valid because alphaL is a GF

# ---------------- 3) Physical parameters ----------------
# Molecular diffusivities
Dil = Parameter(2.0e-9)                        # m^2/s (CO2 in water)
Dig = Parameter(1.5e-5)                        # m^2/s (CO2 in air)

# Henry's constant (dimensionless): k_i = c_ig^* / c_il^*
# For CO2: typically k ~ 0.8-1.2 (gas/liquid concentration ratio at equilibrium)
kH = Parameter(1.0)

# Reaction parameters (CO2–MEA)
T      = Parameter(298.15)                      # K
phiMEA = Parameter(0.30)                        # MEA mole fraction
thetaL = Parameter(0.10)                        # CO2 loading
KCO2   = Parameter(1.0e-5)                      # equilibrium constant
Rgas   = 8.314462618                            # J/(mol K)
cMEA   = Parameter(1000.0)                      # mol/m^3 (MEA concentration)

# Advection velocity (imported flow)
v_adv = u

# ---------------- 4) Derived quantities ----------------

# Harmonic mean diffusivity across phases: Eq. after (19)
# D_i = (α_L * D_il * D_ig) / (α_L * D_ig + (1-α_L) * D_il)
Di = (aL * Dil * Dig) / (aL * Dig + one_m * Dil + tiny)

# Reaction rate constant: Eq. (21)
r = exp(20.54 - 5612.91/T)

# Equilibrium concentration: Eq. (22)
# c_CO2(eq) = (30.96 - 10584/T - 7.187*φ_MEA) * θ²/((1-2θ)²) * 1/(K_CO2*R*T)
theta_term = (thetaL**2) / ((1 - 2*thetaL)**2 + tiny)
c_eq = exp(30.96 - 10584/T - 7.187*phiMEA) * theta_term / (KCO2 * Rgas * T)
# ---------------- 5) Interface drift term Γ ----------------
# From the paper: Γ_i = -D_i * (c_i(1-k_i))/(α_L + k_i(1-α_L)) * ∇α_L
# Note: This is POSITIVE for transport FROM liquid TO gas when k<1
def Gamma_of(c_field):
    denominator = aL + kH * one_m + tiny
    # Fixed sign: should be MINUS as per Eq. (19)
    return -Di * (c_field * (1 - kH)) / denominator * grad_aL

# ---------------- 6) Reaction term W ----------------
# Eq. (20): W_CO2 = r(c_CO2 - c_CO2(eq)) * c_MEA * α_L
# Only active in liquid phase (α_L factor)
def W_of(c_field):
    return r * (c_field - c_eq) * cMEA * aL

# ---------------- 7) Time stepping (semi-implicit) ----------------
dt = 3e-4
c_trial, w = C.TnT()

# Dirichlet BC on Liq_Inlet (e.g., inlet CO2 concentration)
c_in = Parameter(10.0)  # mol/m^3

# Helper for Dirichlet data
gD = GridFunction(C, name="gD")
gD.vec.FV().NumPy()[:] = 0.0
gD.Set(c_in, definedon=mesh.Boundaries("Gas_Inlet"))

# Initial condition
c.vec.FV().NumPy()[:] = 0.0

nsteps = 2000

with TaskManager():
    for step in range(nsteps):
        #print(f"\n=== ADR step {step+1}/{nsteps} ===")

        # Store old solution for explicit terms
        c_old.vec.data = c.vec
        
        # Evaluate explicit terms at old time level
        Gamma_prev = Gamma_of(c_old)
        W_prev     = W_of(c_old)

        # Apply Dirichlet BC (lifting)
        c.Set(gD, definedon=mesh.Boundaries("Gas_Inlet"))

        # Build bilinear form (implicit terms)
        a = BilinearForm(C, symmetric=False)
        L = LinearForm(C)

        # Time derivative: (c^{n+1} - c^n)/dt
        a += (1.0/dt) * c_trial * w * dx
        L += (1.0/dt) * c_old * w * dx

        # Advection: u·∇c (implicit in c)
        a += (v_adv * grad(c_trial)) * w * dx

        # Diffusion: D_i ∇c·∇w (implicit in c)
        a += Di * grad(c_trial) * grad(w) * dx


        #SUPG TERM TO STABILIZE IN HIGH VELOITYR EGIONS
        h = specialcf.mesh_size
        nrm_u = Norm(v_adv)
        tau_supg = h / (2*nrm_u + 1e-12)
        
        # Add SUPG term:
        a += tau_supg * (v_adv * grad(c_trial)) * (v_adv * grad(w)) * dx


        # Interface drift: -∇·Γ → weak form: ∫ Γ·∇w (explicit)
        L += Gamma_prev * grad(w) * dx

        # Reaction: W (explicit, source term)
        L += W_prev * w * dx

        # Assemble system
        a.Assemble()
        L.Assemble()

        # Solve: A*(c^{n+1} - c_D) = b - A*c_D
        # where c_D contains the Dirichlet data
        rhs = L.vec.CreateVector()
        rhs.data = L.vec - a.mat * c.vec

        # Solve on free DOFs
        inv = a.mat.Inverse(C.FreeDofs())
        c.vec.data += inv * rhs

        # Clip negatives in place
        arr = c.vec.FV().NumPy()
        arr[arr < 0.0] = 0.0   # modifies c.vec directly
        arr[arr > 10.0] = 10.0   # modifies c.vec directly
        # Now c is guaranteed nonnegative
        c_min = np.min(arr)
        c_max = np.max(arr)
        c_mean = Integrate(c, mesh) / Integrate(1, mesh)
        #print(f"  c_CO2: min={c_min:.6e}, max={c_max:.6e}, mean={c_mean:.6e}")

        if step % 100 == 0:
            print(step)
print("\nSimulation complete!")

# Optional visualization
# Draw(alphaL, mesh, "alphaL")
# Draw(u, mesh, "velocity")
# Draw(c, mesh, "c_CO2")


Snapshots: 150   per-snapshot DOFs: 174723
Vel DOFs: 116482   Press DOFs: 14789
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900

Simulation complete!


In [3]:
U_data.shape

(150, 174723)

In [4]:
step

1999

In [5]:
from ngsolve.webgui import Draw
Draw(c)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [6]:
Draw(u)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene